In [1]:
from lifelines.datasets import load_rossi
from lifelines import CoxPHFitter

In [3]:
%matplotlib inline
import os
from os import path
from glob import glob
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from utils import *

In [14]:
# cancer_type = 'Lung'
# cluster_num = 3
# rand_run = 4
# clinical_filename = '/home/tunthey/Dropbox/Tunde/analysis/new_datasets/' + cancer_type + '/' + cancer_type +  '_clinical_data.txt'
# drug_filename = '/home/tunthey/Dropbox/Tunde/analysis/new_datasets/' + cancer_type + '/' + cancer_type +  '_drug_data.txt'
# cluster_label = '/home/tunthey/Dropbox/Tunde/analysis/new_results/MVKKM_newdata/labels/' + cancer_type + '_mvkkm_labels_K' +  str(cluster_num) + '_random_init_run' + str(rand_run) + '_rbf_kernel_old_gamma_with_shankar_distance_second_derv.txt'
# pam50_file = '/home/tunthey/Dropbox/Tunde/analysis/PAM50/bioclassifier_R/Breastcancerpan50subtyperesult_pam50scores.txt'


In [34]:
# cluster_l = open(cluster_label).readlines()
# cluster_list = [s.rstrip() for s in cluster_l]
# np.array(cluster_list)
# cluster_list

In [4]:
# rossi_dataset = load_rossi()
# cph = CoxPHFitter()
# cph.fit(rossi_dataset, duration_col='week', event_col='arrest')

# cph.plot()


In [11]:
with open('../data/exp_final_gene_list.txt', 'r') as f:
    final_genes= [l.rstrip() for l in f.readlines()]
with open('../data/hint_inters_pan.txt', 'r') as f:
    hint_u_pan= [l.rstrip() for l in f.readlines()]

In [5]:
cancer_types = ['BLCA','BRCA','CRC', 'GBM', 'HNSC', 'KIRC', 'LAML', 'LUAD', 'LUSC', 'OV', 'UCEC']

In [6]:
comps = []
significant_cancers =[]
with open('../hint/out/cancer_subtype_test_results/mutex_t07_nsep_cov_nsep/cc_n100_cancer_subtype_tests.txt', 'r') as f:
    lines = f.readlines()[:]
    for i in range(0,len(lines), 12):
        comp = lines[i].strip().split('[')[-1].split(',')
        comp = [c.replace(']', '').replace('\'', '').strip() for c in comp]
        comps.append(comp)
        type_interest = [j-1 for j in range(1,12) if len(lines[i+j].split()) ==8]
        significant_cancers.append(type_interest)
        

In [25]:
cancer2modules = {}
for i,m in enumerate(comps):
    for t in significant_cancers[i]:
        if cancer_types[t] in cancer2modules.keys():
            cancer2modules[cancer_types[t]].append(m)
        else:
            cancer2modules[cancer_types[t]] = [m]

In [50]:
module2id = {str(m):i for i,m in enumerate(comps)}
id2module = {i:str(m) for i,m in enumerate(comps)}

In [143]:
cancer2modules.keys()

dict_keys(['HNSC', 'LAML', 'LUAD', 'KIRC', 'OV', 'BRCA', 'GBM', 'LUSC', 'UCEC', 'CRC'])

In [57]:
id2module[0]

"['POLR2A', 'SMARCA2', 'SMARCA4']"

In [59]:
out_path = '../hint/out/'
data_path = '../data/'
data_files = 'pancancer/'
clinic_path = data_path+'clinical_data/'
clinic_file = '{}_clinical_data.txt'
files = glob(data_path+data_files+'*/[!MANIFEST]*.txt')

In [9]:
cancer2file = {c.split('/')[-1].split('.')[0]:c for c in files}

In [208]:
df = pd.read_csv(cancer2file['GBM'], delimiter = '\t', low_memory=False)
df.drop(df.index[0], inplace=True)
gene_and_id = df.iloc[:, 0].values
df.index = final_genes
df.drop(df.columns[0], axis=1, inplace= True)
df = df.loc[list(hint_u_pan)]

In [250]:
df_ = df.T
df_.index.names = ['patients']
df_.columns.name = 'genes'

In [230]:
ids= [s.split('-')[3] for s in df_.index]

In [231]:
np.unique(ids, return_counts=True)

(array(['01A', '01B', '01C', '02A', '02B', '11A'], dtype='<U3'),
 array([142,  10,   1,  12,   1,   5]))

In [251]:
keep_patient = [x.split('-')[3][:2] == '01' for x in df_.index]
df_ = df_.loc[keep_patient]

In [252]:
df_.shape

(153, 9523)

In [253]:
df_.index = ['-'.join(x.split('-')[:3]) for x in df_.index]
df_.index.names = ['patients']

In [254]:
df2.head()

,event,time_to_event,NUP98,ZKSCAN5,LRRC7,RHO,WASH3P,KIF21B,LASP1,C2orf62,...,LCA5,CACNB2,AMPH,GNG10,COMTD1,KNG1,RRAGB,RRAGC,TMED2,TRPT1
patients,,,,,,,,,,,,,,,,,,,,,
TCGA-06-5408,0,357.0,957.7029,877.3054,3.5340,1.7670,417.4489,469.1331,4161.2369,1.7670,...,99.8343,203.6444,397.5704,879.9558,55.2181,0.0000,523.4677,454.5555,3897.0734,441.7449
TCGA-06-5410,0,108.0,1427.7678,522.1171,29.1486,1.0228,323.1910,226.0291,4663.2575,14.8300,...,122.7308,171.8231,265.4053,2481.7182,71.0816,0.0000,313.9862,587.1542,4801.8410,407.5684
TCGA-06-5411,0,254.0,969.1529,658.3794,159.3002,0.9208,461.7864,1632.1363,3436.0037,1.8416,...,192.4494,410.6814,2476.9797,1406.5378,68.1400,0.4604,925.8748,506.2569,4277.6243,494.0147
TCGA-06-5412,0,138.0,1286.6985,568.0975,17.4881,1.5898,387.3874,308.9560,3826.1791,5.8294,...,95.9194,146.2639,115.5273,1550.0477,45.0450,0.0000,369.8993,486.2957,5172.7610,565.9777
TCGA-06-5413,1,268.0,1401.2230,838.1938,6.5851,1.4111,405.9266,945.9078,4133.1138,23.5183,...,140.6397,8.4666,563.4995,1268.7441,33.3960,0.0000,413.9229,460.4892,4723.4243,440.2634


In [255]:
df2 = df_clinic.join(df_)

In [256]:
df2 = df2.loc[~df2[df2.columns[2:]].isnull().any(axis=1)]

In [257]:
df2.head()

,event,time_to_event,age,gender,NUP98,ZKSCAN5,LRRC7,RHO,WASH3P,KIF21B,...,LCA5,CACNB2,AMPH,GNG10,COMTD1,KNG1,RRAGB,RRAGC,TMED2,TRPT1
patients,,,,,,,,,,,,,,,,,,,,,
TCGA-06-5408,0,357.0,54,1,957.7029,877.3054,3.5340,1.7670,417.4489,469.1331,...,99.8343,203.6444,397.5704,879.9558,55.2181,0.0000,523.4677,454.5555,3897.0734,441.7449
TCGA-06-5410,0,108.0,72,1,1427.7678,522.1171,29.1486,1.0228,323.1910,226.0291,...,122.7308,171.8231,265.4053,2481.7182,71.0816,0.0000,313.9862,587.1542,4801.8410,407.5684
TCGA-06-5411,0,254.0,51,0,969.1529,658.3794,159.3002,0.9208,461.7864,1632.1363,...,192.4494,410.6814,2476.9797,1406.5378,68.1400,0.4604,925.8748,506.2569,4277.6243,494.0147
TCGA-06-5412,0,138.0,78,1,1286.6985,568.0975,17.4881,1.5898,387.3874,308.9560,...,95.9194,146.2639,115.5273,1550.0477,45.0450,0.0000,369.8993,486.2957,5172.7610,565.9777
TCGA-06-5413,1,268.0,67,0,1401.2230,838.1938,6.5851,1.4111,405.9266,945.9078,...,140.6397,8.4666,563.4995,1268.7441,33.3960,0.0000,413.9229,460.4892,4723.4243,440.2634


In [258]:
df2.shape

(151, 9527)

In [206]:
for module in cancer2modules['LAML']:

        module_id = module2id[str(module)]
#         if not os.path.exists(results_path+'{}/'.format(module_id)):
#             os.mkdir(results_path+'{}/'.format(module_id))
#         csv_path = results_path+'{}/{}.csv'.format(module_id, cancer)
        df2 = df2.astype('float')
        tmp_df  = df2[module+['event','time_to_event']]
        cph = CoxPHFitter()
        cph.fit(tmp_df, duration_col='time_to_event', event_col='event', show_progress=False)
        s = cph.summary

In [274]:
clinical_data = clinic_file.format('GBM')
df_clinic = pd.read_csv(clinic_path+'nationwidechildrens.org_clinical_follow_up_v1.0_gbm.txt', delimiter = '\t', low_memory=False)


In [275]:
df_clinic = df_clinic.loc[2:].reset_index().drop('index', axis=1)

In [ ]:
df_clinic['patients'] = df_clinic['bcr_patient_barcode'].apply()

In [ ]:
df_clinic['age']

In [ ]:
df_clinic['gender']
df_clinic['event']
df_clinic['time_to_event']

In [247]:
df_clinic = df_clinic.set_index('patient_id')
df_clinic.index.names = ['patients']
#df_clinic.set_index('patient_id')

In [248]:
df_clinic =df_clinic[~df_clinic.isnull().any(axis=1)]

In [249]:
df_clinic.shape

(593, 4)

In [276]:
df_clinic.head()

,bcr_patient_uuid,bcr_patient_barcode,bcr_followup_barcode,bcr_followup_uuid,form_completion_date,followup_reason,followup_lost_to,radiation_treatment_adjuvant,pharmaceutical_tx_adjuvant,treatment_outcome_first_course,vital_status,last_contact_days_to,death_days_to,tumor_status,new_tumor_event_dx_indicator,treatment_outcome_at_tcga_followup,ecog_score,karnofsky_score,performance_status_timing
0,30a1fe5e-5b12-472c-aa86-c2db8167ab23,TCGA-02-0001,TCGA-02-0001-F22169,59ed9ea3-93d3-4fa4-b186-280698d1cebf,2008-12-16,[Not Available],[Not Available],YES,YES,[Not Available],Dead,279,358,WITH TUMOR,[Not Available],[Not Available],[Not Available],80,[Not Available]
1,df3c1d61-79c1-43e9-971a-8029497ffeab,TCGA-02-0003,TCGA-02-0003-F22176,e9f7905a-d445-4f84-b80c-33bf6abeba92,2008-12-16,[Not Available],[Not Available],YES,YES,[Not Available],Dead,144,144,WITH TUMOR,[Not Available],[Not Available],[Not Available],100,[Not Available]
2,3ac8f759-2df0-4ef5-913f-ce05b8dd8937,TCGA-02-0004,TCGA-02-0004-F22180,b1cc8f47-2ca6-4d98-a461-ab6582e06b97,2008-12-17,[Not Available],[Not Available],YES,YES,[Not Available],Dead,345,345,WITH TUMOR,[Not Available],[Not Available],[Not Available],80,[Not Available]
3,98714d95-b62e-4f34-9cd1-91542da463eb,TCGA-02-0006,TCGA-02-0006-F22185,679559e4-a79a-42d0-80ff-e7c637f5cc28,2008-12-17,[Not Available],[Not Available],YES,YES,[Not Available],Dead,558,558,WITH TUMOR,[Not Available],[Not Available],[Not Available],80,[Not Available]
4,fb75eb9d-05be-4e6b-a515-f39efb15e66c,TCGA-02-0007,TCGA-02-0007-F22191,b4a8866b-4d60-428a-87ce-3a294fe7a446,2008-12-16,[Not Available],[Not Available],YES,YES,[Not Available],Dead,705,705,WITH TUMOR,[Not Available],[Not Available],[Not Available],80,[Not Available]


In [101]:
df_clinic.shape

(1097, 9)

In [40]:
df1 = df_.astype('float')

In [259]:
tmp_df  = df2[['NOTCH1']+['age','gender','event','time_to_event']].astype('float')
cph = CoxPHFitter()
cph.fit(tmp_df, duration_col='time_to_event', event_col='event', show_progress=False)

<lifelines.CoxPHFitter: fitted with 151 observations, 99 censored>

In [262]:
s = cph.summary

In [260]:
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 151 observations, 99 censored>
      duration col = 'time_to_event'
         event col = 'event'
number of subjects = 151
  number of events = 52
    log-likelihood = -215.92
  time fit was run = 2019-01-23 08:51:28 UTC

---
        coef  exp(coef)  se(coef)     z    p  log(p)  lower 0.95  upper 0.95   
NOTCH1  0.00       1.00      0.00  0.38 0.70   -0.35       -0.00        0.00   
age     0.00       1.00      0.01  0.07 0.94   -0.06       -0.02        0.02   
gender -0.11       0.90      0.31 -0.35 0.72   -0.32       -0.71        0.49   
---
Signif. codes: 0 '***' 0.0001 '**' 0.001 '*' 0.01 '.' 0.05 ' ' 1

Concordance = 0.49
Likelihood ratio test = 0.28 on 3 df, log(p)=-0.04


In [263]:
s

,coef,exp(coef),se(coef),z,p,log(p),lower 0.95,upper 0.95
NOTCH1,0.000028,1.000028,0.000074,0.382534,0.702065,-0.353729,-0.000117,0.000174
age,0.000814,1.000814,0.011198,0.072676,0.942064,-0.059682,-0.021134,0.022761
gender,-0.108387,0.897281,0.307786,-0.352149,0.724727,-0.321961,-0.711636,0.494863
